# Memory

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [42]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        super().__init__()
        self.messages = []

    def add_message(self, message: BaseMessage):
        """Add a sigle message to the store!!!"""
        self.messages.append(message)

    def clear(self):
        self.messages = []

    def __repr__(self):     # __repr__ 메서드는 객체의 상태를 문자열로 표현할 때 사용되는 메서드 : 디버깅 목적으로 사용
        # return f'{self.__class__.__name__}(\n{self.messages}\n)'
        return str(self.messages)

In [43]:
store = {}    # item(key=session_id, value=InMemoryHistory 인스턴스)

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()

    return store[session_id]

In [44]:
from langchain_core.messages import HumanMessage, AIMessage

# step1
history_test = get_by_session_id('test')
history_test.add_message(HumanMessage(content='hello'))
history_test.add_message(HumanMessage(content='goodbye'))
history_test.add_message(HumanMessage(content='how are you?'))

history_test

[HumanMessage(content='hello', additional_kwargs={}, response_metadata={}), HumanMessage(content='goodbye', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={})]

In [45]:
# step2
history_test = get_by_session_id('user1')
history_test.add_message(HumanMessage(content='인간메시지, 커널이 유지되고 있어서 instance가 유지되고 있어요.'))

history_test

[HumanMessage(content='인간메시지, 커널이 유지되고 있어서 instance가 유지되고 있어요.', additional_kwargs={}, response_metadata={})]

In [46]:
# 메모리를 저장한 것 처럼 store에 기억한다. → kernel이 유지되고 있어서 instance가 유지되고 있다.
store    # step1에서 저장한 것이 있고, step2에서 저장한 것이 있다.

{'test': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}), HumanMessage(content='goodbye', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={})],
 'user1': [HumanMessage(content='인간메시지, 커널이 유지되고 있어서 instance가 유지되고 있어요.', additional_kwargs={}, response_metadata={})]}

In [47]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill} 전문가 AI 어시스턴트야.'),
    MessagesPlaceholder(variable_name='history'),
    ('human', '{query}')
])

model = ChatOpenAI(model='gpt-4o-mini', temperature=0.5)
chain = prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'
)

In [48]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '토끼는 농장에서 나무를 세 그루 키우고 있습니다.'},
    config={'configurable': {'session_id': 'rabbit'}}
)

print(response)


content='토끼가 농장에서 나무를 세 그루 키우고 있다니 귀엽네요! 어떤 종류의 나무를 키우고 있나요? 또는 나무를 키우는 데 어떤 특별한 이유가 있을까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 40, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYRfpDJCXg4Oe9VJEFONPEjYQzilP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--94aad5bd-9979-492c-9bfb-aff122ba4fe4-0' usage_metadata={'input_tokens': 40, 'output_tokens': 49, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [49]:
store

{'test': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}), HumanMessage(content='goodbye', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={})],
 'user1': [HumanMessage(content='인간메시지, 커널이 유지되고 있어서 instance가 유지되고 있어요.', additional_kwargs={}, response_metadata={})],
 'rabbit': [HumanMessage(content='토끼는 농장에서 나무를 세 그루 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='토끼가 농장에서 나무를 세 그루 키우고 있다니 귀엽네요! 어떤 종류의 나무를 키우고 있나요? 또는 나무를 키우는 데 어떤 특별한 이유가 있을까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 40, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fi

In [50]:
input_session_id = 'Haward'
input_query = '내 이름은 Haward, 현재 미국에서 대학교 생활을 하고 있고 컴퓨터 공학과를 전공하고 있어.'
input_skill = '진로 설계'

response = chain_with_history.invoke(
    {'skill': input_skill, 'query': input_query},
    config={'configurable': {'session_id': input_session_id}}
)
print(response.content)

안녕하세요, Haward! 컴퓨터 공학을 전공하고 계시다니 멋지네요. 어떤 분야에 관심이 있으신가요? 예를 들어, 소프트웨어 개발, 인공지능, 사이버 보안, 데이터 과학 등 다양한 분야가 있습니다. 또한, 진로에 대한 고민이나 궁금한 점이 있다면 말씀해 주세요. 도움이 될 수 있도록 최선을 다하겠습니다!


In [51]:
store

{'test': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}), HumanMessage(content='goodbye', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={})],
 'user1': [HumanMessage(content='인간메시지, 커널이 유지되고 있어서 instance가 유지되고 있어요.', additional_kwargs={}, response_metadata={})],
 'rabbit': [HumanMessage(content='토끼는 농장에서 나무를 세 그루 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='토끼가 농장에서 나무를 세 그루 키우고 있다니 귀엽네요! 어떤 종류의 나무를 키우고 있나요? 또는 나무를 키우는 데 어떤 특별한 이유가 있을까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 40, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fi

In [52]:
# 두 번째 대화 (같은 세션!)
input_session_id = 'Haward'  # ← 같은 세션
input_query = 'Haward의 전공을 살려서 취업할 수 있는 미국 회사는?'
response = chain_with_history.invoke(
    {'skill': '질의응답', 'query': input_query},
    config={'configurable': {'session_id': input_session_id}}
)
print(response.content)

컴퓨터 공학 전공으로 미국에서 취업할 수 있는 회사는 매우 다양합니다. 다음은 몇 가지 주요 분야와 회사들입니다:

1. **기술 대기업**:
   - **Google**: 소프트웨어 개발, 데이터 분석, 인공지능 등 다양한 직무가 있습니다.
   - **Microsoft**: 클라우드 컴퓨팅, 소프트웨어 개발, 사이버 보안 등.
   - **Apple**: 하드웨어 및 소프트웨어 개발, UI/UX 디자인 등.
   - **Amazon**: AWS(아마존 웹 서비스), 소프트웨어 개발, 데이터 과학 등.

2. **스타트업**:
   - 많은 혁신적인 스타트업들이 컴퓨터 공학 전공자를 찾고 있습니다. 예를 들어, **Stripe**, **Airbnb**, **Uber**와 같은 회사들은 기술 인재를 많이 필요로 합니다.

3. **금융 및 핀테크**:
   - **Goldman Sachs**, **JP Morgan**, **Square**와 같은 금융 기관이나 핀테크 회사들도 소프트웨어 개발자, 데이터 분석가 등을 채용합니다.

4. **게임 개발**:
   - **Electronic Arts**, **Activision Blizzard**, **Epic Games**와 같은 게임 회사들도 컴퓨터 공학 전공자를 많이 필요로 합니다.

5. **헬스케어 및 바이오테크**:
   - **Johnson & Johnson**, **Pfizer**, **Medtronic** 같은 회사에서는 헬스케어 IT 솔루션 개발, 데이터 분석 등에 많은 기회를 제공합니다.

6. **자동차 산업**:
   - **Tesla**, **Ford**, **General Motors**와 같은 자동차 제조업체들도 자율주행 기술, 소프트웨어 개발 등에서 컴퓨터 공학 전공자를 채용하고 있습니다.

이 외에도 다양한 산업에서 컴퓨터 공학 전공자를 필요로 하므로, 본인의 관심 분야와 연관된 회사를 찾아보는 것이 좋습니다. 인턴십이나 프로젝트 경험을 통해 이력서를 강화하는 것도 중요합니다. 도움이

In [53]:
store

{'test': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}), HumanMessage(content='goodbye', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={})],
 'user1': [HumanMessage(content='인간메시지, 커널이 유지되고 있어서 instance가 유지되고 있어요.', additional_kwargs={}, response_metadata={})],
 'rabbit': [HumanMessage(content='토끼는 농장에서 나무를 세 그루 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='토끼가 농장에서 나무를 세 그루 키우고 있다니 귀엽네요! 어떤 종류의 나무를 키우고 있나요? 또는 나무를 키우는 데 어떤 특별한 이유가 있을까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 40, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fi

In [56]:
# 세번째 대화 (같은 세션!)
input_session_id = 'Haward'  # ← 같은 세션
input_query = 'Haward의 생활 패턴을 고려해서 추천해줄 취미는?'
input_skill = '추천시스템'
response = chain_with_history.invoke(
    {'skill': input_skill, 'query': input_query},
    config={'configurable': {'session_id': input_session_id}}
)
print(response.content)

Haward의 생활 패턴을 고려할 때, 대학교 생활과 컴퓨터 공학 전공을 감안하여 몇 가지 취미를 추천해 드릴게요:

1. **코딩 및 프로젝트 개발**:
   - 개인 프로젝트를 통해 새로운 기술을 배우거나 오픈 소스 프로젝트에 참여해 보세요. GitHub에서 다른 개발자들과 협업하는 것도 좋은 경험이 될 수 있습니다.

2. **해커톤**:
   - 다양한 해커톤에 참여하여 팀과 함께 문제를 해결하고, 새로운 아이디어를 실현해보세요. 이는 네트워킹 기회도 제공하고, 실력을 키울 수 있는 좋은 방법입니다.

3. **게임 개발**:
   - Unity나 Unreal Engine과 같은 게임 엔진을 사용하여 간단한 게임을 만들어보세요. 게임 개발은 창의력과 기술을 동시에 활용할 수 있는 재미있는 취미입니다.

4. **데이터 분석 및 머신러닝 프로젝트**:
   - Kaggle과 같은 플랫폼에서 데이터 분석 대회에 참여하거나, 머신러닝 모델을 만들어보는 것도 좋은 취미입니다. 실제 데이터를 다루며 실력을 쌓을 수 있습니다.

5. **블로그나 유튜브 채널 운영**:
   - IT 관련 주제로 블로그를 쓰거나 유튜브 채널을 운영해보세요. 자신이 배운 것을 다른 사람들과 공유하는 것도 좋은 경험이 될 수 있습니다.

6. **독서**:
   - 기술 서적이나 관련 논문을 읽는 것도 좋습니다. 새로운 트렌드나 기술에 대한 이해를 높일 수 있습니다.

7. **운동 및 야외 활동**:
   - 컴퓨터 공학 전공자는 종종 장시간 앉아있는 경우가 많으므로, 정기적으로 운동을 하거나 야외 활동을 통해 스트레스를 해소하는 것도 중요합니다. 조깅, 자전거 타기, 하이킹 등을 고려해 보세요.

8. **온라인 강의 수강**:
   - Coursera, edX, Udacity 등의 플랫폼에서 새로운 기술이나 언어를 배우는 것도 좋은 취미입니다. 관심 있는 분야의 강의를 찾아보세요.

이 중에서 본인의 관심사와 라이프스타일에 맞는 취미를 선택해 보세요!


In [54]:
input_session_id = 'Jay'
input_query = '내 이름은 Jay, 내 친구 Haward의 전공을 살려서 미국에 취업할 수 있는 회사가 어디 있을까?'
input_skill = '질의응답'

response = chain_with_history.invoke(
    {'skill': input_skill, 'query': input_query},
    config={'configurable': {'session_id': input_session_id}}
)
print(response.content)

Haward의 전공에 따라 미국에서 취업할 수 있는 회사는 다양합니다. 전공에 따라 추천할 수 있는 회사가 달라지므로, Haward의 전공을 알려주시면 보다 구체적인 정보를 제공해드릴 수 있습니다. 예를 들어, 컴퓨터 공학, 생명과학, 경영학 등 전공에 따라 적합한 기업이 다를 수 있습니다. 전공을 알려주시면 더 자세한 도움을 드릴 수 있습니다!


같은 session_id를 사용하면 같은 대화 기록을 유지할 수 있다.<br>
**하지만, 다른 session_id를 사용하면 다른 대화 기록을 유지할 수 있다.**

In [55]:
store

{'test': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}), HumanMessage(content='goodbye', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={})],
 'user1': [HumanMessage(content='인간메시지, 커널이 유지되고 있어서 instance가 유지되고 있어요.', additional_kwargs={}, response_metadata={})],
 'rabbit': [HumanMessage(content='토끼는 농장에서 나무를 세 그루 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='토끼가 농장에서 나무를 세 그루 키우고 있다니 귀엽네요! 어떤 종류의 나무를 키우고 있나요? 또는 나무를 키우는 데 어떤 특별한 이유가 있을까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 40, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fi